<a href="https://colab.research.google.com/github/KanishkThamman/Face-Req-Using-FaceNet/blob/main/CV_Student_identification_model_V1__final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#download and import libs

In [ ]:
#cloning a git repo and intalling other things. Getting a sample dataset with the req.txt...
!pip install gdown==4.4.0
#!git clone https://github.com/KanishkThamman/Dataset
!curl -o 'haarcascade_frontalface_default.xml' https://raw.githubusercontent.com/GeekyShiva/OpenCV-miniProject-FaceDetection/master/haarcascade_frontalface_default.xml
!gdown --folder https://drive.google.com/drive/folders/1pwQ3H4aJ8a6yyJHZkTwtjcL4wYWQb7bn
!gdown --folder https://drive.google.com/drive/folders/1TX-IkAkmMq46L35r2tunTYa8FtFrWd49?usp=sharing
!pip install -r requirements.txt



In [ ]:
#import the lib...
import os
from os import listdir
from PIL import Image
from numpy import asarray
from numpy import expand_dims
from matplotlib import pyplot
from keras.models import load_model
import numpy as np
import pickle
import cv2
import shutil

In [ ]:
#unziping the sample dataset
shutil.unpack_archive('Dataset/photos.zip', 'Dataset') #the sample dataset(per person one pic is only req )
os.remove("Dataset/photos.zip") #remove the zip file 

Make the  Model Identify You By Training it on your picture (optional)
directions of use :-
after running the code a window called photoclicker will pop up and you will have to click the Spacebar once to click your photo 

In [ ]:
print("Your Name?")
name = str(input())
cam = cv2.VideoCapture(0)
 
cv2.namedWindow("Photo_Clicker")
 
img_counter = 0
 
while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    cv2.imshow("test", frame)
 
    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed
        while img_counter <= 0 :
          img_name = "Dataset/photos/"+name+".png"
          cv2.imwrite(img_name, frame)
          print("done!!")
          img_counter += 1
        else :
          break
cam.release()
 
cv2.destroyAllWindows()

# making the keys for the database (training the model )

In [ ]:
# load the model
MyFaceNet = load_model('keras-facenet/model/facenet_keras.h5')
HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile('haarcascade_frontalface_default.xml'))
# summarize input and output shape
print(MyFaceNet.inputs)
print(MyFaceNet.outputs)

In [ ]:
#Makaing the key for identification of a person

folder = r"Dataset/photos/"
database = {}
for filename in listdir(folder):

    path = folder + filename
    gbr1 = cv2.imread(folder +filename)
    cascade = cv2.CascadeClassifier()
    
    face = HaarCascade.detectMultiScale(gbr1,1.1,4)
    
    if len(face)>0:
        x1, y1, width, height = face[0]         
    else:
        x1, y1, width, height = 1, 1, 10, 10
        
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    
    gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
    gbr = Image.fromarray(gbr)                  
    gbr_array = asarray(gbr)
    
    face = gbr_array[y1:y2, x1:x2]                        
    
    face = Image.fromarray(face)                       
    face = face.resize((160,160))
    face = asarray(face)
    
    face = face.astype('float32')
    mean, std = face.mean(), face.std()
    face = (face - mean) / std
    
    face = expand_dims(face, axis=0)
    signature = MyFaceNet.predict(face)
    
    database[os.path.splitext(filename)[0]]=signature

In [ ]:
myfile = open("data.pkl", "wb")
pickle.dump(database, myfile)
myfile.close()

Preidiction for the person

In [ ]:
myfile = open("data.pkl", "rb")
database = pickle.load(myfile)
myfile.close()

In [ ]:

#turing on the camara for pridiction
cap = cv2.VideoCapture(0)
cv2.namedWindow("Photo_Clicker")
while(True):

    _, gbr1 = cap.read()
    
    face = HaarCascade.detectMultiScale(gbr1,1.1,4)
    
    if len(face)>0:
        x1, y1, width, height = face[0]        
    else:
        x1, y1, width, height = 1, 1, 10, 10
    
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    
    
    gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
    gbr = Image.fromarray(gbr, 'RGB')                 
    gbr_array = asarray(gbr)
    
    face = gbr_array[y1:y2, x1:x2]                        
    
    face = Image.fromarray(face)                       
    face = face.resize((160,160))
    face = asarray(face)
    
    face = face.astype('float32')
    mean, std = face.mean(), face.std()
    face = (face - mean) / std
    
    face = expand_dims(face, axis=0)
    signature = MyFaceNet.predict(face)
    
    min_dist=100
    identity=' '
    #compareing the keys genrated from the above video to compare with the database var created above 
    for key, value in database.items() :
        dist = np.linalg.norm(value-signature)
        if dist < min_dist:
            min_dist = dist
            identity = key
            
    cv2.putText(gbr1,identity, (100,100),cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
    cv2.rectangle(gbr1,(x1,y1),(x2,y2), (0,255,0), 2)
        
    cv2.imshow('res',gbr1)
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
        
cv2.destroyAllWindows()
cap.release()